In [2]:
# from traj_len_15 import DeepAccidentDataset
from data.Safe2Unsafe import DeepAccidentDataset
from method.ViT import InDCBFTrainer, InDCBFController, Barrier
import torch
import time
import pytorch_lightning as pl

In [5]:
data = DeepAccidentDataset(train_batch_size=32,val_batch_size=32,num_workers=16)
data.setup()
train_dataloader = data.train_dataloader()
test_dataloader = data.val_dataloader()
barrier = Barrier(2,512)
model = InDCBFController(2,"cuda",model="google/vit-base-patch16-224",latent_dim=512)
checkpoint = torch.load("/root/tf-logs/BarrierClassifier/version_1/checkpoints/epoch=80-step=16037.ckpt")
trainer = InDCBFTrainer(model,barrier)
trainer.load_state_dict(checkpoint['state_dict'])

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [6]:
for idx, (i_all,u_all,label) in enumerate(test_dataloader):
    if label.sum()>0:
        break

In [15]:
x_init = torch.zeros(1,512).to('cuda')
model = trainer.model.to("cuda")
barrier = trainer.barrier.to("cuda")
print(label)

tensor([[1],
        [2],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])


In [35]:
imgs = i_all[10].to("cuda")
us = u_all[10].to("cuda")
x_p = model.encoder(imgs[0,:].unsqueeze(0),x_init,us[0].unsqueeze(0))
for i in range(1,15):
# u, result = model(i[0,1].unsqueeze(0),u[0,0].unsqueeze(0),x_p,u[0,0].unsqueeze(0),barrier)
    v, result, prob = model(imgs[i].unsqueeze(0),us[i-1].unsqueeze(0),x_p,us[i].unsqueeze(0),barrier)
    x_p = model.encoder(imgs[i,:].unsqueeze(0),x_p,us[i-1].unsqueeze(0))
    print(v.value,us[i])
# print(us[0])
# print(us[1])

[ 0.0508374  -4.17871857] tensor([ 0.0508, -4.1787], device='cuda:0')
[ 0.04569366 -3.84402585] tensor([ 0.0457, -3.8440], device='cuda:0')
[ 0.04593947 -3.83066034] tensor([ 0.0459, -3.8307], device='cuda:0')
[ 0.04822314 -3.99490833] tensor([ 0.0482, -3.9949], device='cuda:0')


IndexError: index 5 is out of bounds for dimension 0 with size 5

In [29]:
print(v.value)
print(us[10])

[-2506.46806297  -386.10911042]


IndexError: index 10 is out of bounds for dimension 0 with size 5

In [37]:
u_ref = us[1].unsqueeze(0)
u_p = us[1].unsqueeze(0)
i = imgs[1].unsqueeze(0)
import cvxpy as cp
u_ref = u_ref.view(-1).cpu().numpy()
x = model.encoder(i,x_p,u_p)
f, g = model.ode(x)
f = f.view(-1).detach().cpu().numpy()
g = g.view(-1,2).detach().cpu().numpy()
b = barrier(x)
d_b = torch.autograd.grad(b,x,retain_graph=True)[0]
b = b.view(-1).detach().cpu().numpy()
x = x.view(-1).detach().cpu().numpy()
d_b = d_b.view(-1).cpu().numpy()
u = cp.Variable(u_ref.shape)
t1 = d_b @ f
t2 = d_b @ g 
t3 = b/15
objective = cp.Minimize(cp.sum_squares(u - u_ref))
constraints = [(t1+t2@u+t3)>=0]
prob = cp.Problem(objective, constraints)
result = prob.solve()
t1, t2, t3

(0.000443073,
 array([ 0.00862157, -0.03043856], dtype=float32),
 array([0.0392131], dtype=float32))

(0.0023901234,
 array([-0.00282371, -0.01853349], dtype=float32),
 array([-0.0297253], dtype=float32))

In [7]:
b_all = []
label_all = []
for idx, (i,u,label) in enumerate(test_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = model.simulate(i,u,dt=0.1,window_size=1,rtol=5e-6)
    b = barrier(x).squeeze(-1)
    b_all.append(b.cpu())
    label_all.append(label.squeeze(-1))

100%|██████████| 14/14 [00:02<00:00,  4.70it/s]


In [12]:
import numpy as np
bs = torch.cat(b_all)
labels = torch.cat(label_all)
results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results.txt",results)

In [43]:
b = results[:,:-1]
label = results[:,-1]
mask = np.zeros_like(b)
mask[label == 1,-5:] = 1
b_safe = b[label == 0]
b_unsafe = b[label == 1,-5:]

In [44]:
max_safe = b_safe.max()
min_safe = b_safe.min()
mean_safe = b_safe.mean()
std_safe = b_safe.std()
max_unsafe = b_unsafe.max()
min_unsafe = b_unsafe.min()
mean_unsafe = b_unsafe.mean()
std_unsafe = b_unsafe.std()
print("Safe", max_safe, min_safe, mean_safe, std_safe)
print("Unsafe", max_unsafe, min_unsafe, mean_unsafe, std_unsafe)

Safe 1.0 -0.67952687 0.97396356 0.14432254
Unsafe 1.0 -0.6665393 0.91301495 0.343813


In [4]:
b_all_train = []
label_all_train = []
for idx, (i,u,label) in enumerate(train_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = model.simulate(i,u,dt=0.1,window_size=1,rtol=5e-6)
    b = barrier(x).squeeze(-1)
    b_all_train.append(b.cpu())
    label_all_train.append(label.squeeze(-1))

100%|██████████| 14/14 [00:02<00:00,  4.69it/s]


In [5]:
import numpy as np
bs_train = torch.cat(b_all_train)
labels_train = torch.cat(label_all_train)
results_train = torch.cat([bs_train,labels_train.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_train.txt",results_train)

In [6]:
b = results_train[:,:-1]
label = results_train[:,-1]
mask = np.zeros_like(b)
mask[label == 1,-5:] = 1
b_safe = b[label == 0]
b_unsafe = b[label == 1,-5:]

In [7]:
max_safe = b_safe.max()
min_safe = b_safe.min()
mean_safe = b_safe.mean()
std_safe = b_safe.std()
max_unsafe = b_unsafe.max()
min_unsafe = b_unsafe.min()
mean_unsafe = b_unsafe.mean()
std_unsafe = b_unsafe.std()
print("Safe", max_safe, min_safe, mean_safe, std_safe)
print("Unsafe", max_unsafe, min_unsafe, mean_unsafe, std_unsafe)

Safe 1.0 -0.60138047 0.9851279 0.0887461
Unsafe 0.9999999 -0.7876854 0.76094234 0.52417785
